In [1]:
def sum_weights_for_unique_masks(masks, masks_weights, logits): #, poolers):
    # Convert each boolean mask to a unique string of 0s and 1s
    mask_strs = [''.join(map(str, mask.bool().int().flatten().tolist())) for mask in masks]
    img_size = 66

    # Dictionary to store summed weights for each unique mask
    unique_masks_weights = {}
    unique_masks_logits = {}
    unique_masks_count = {}
    unique_masks_dict = {}

    for i, (mask_str, weight, pred) in enumerate(zip(mask_strs, masks_weights, logits)):
        if mask_str in unique_masks_weights:
            unique_masks_weights[mask_str] += weight
            unique_masks_logits[mask_str] += pred
            unique_masks_count[mask_str] += 1
        else:
            unique_masks_dict[mask_str] = masks[i]
            unique_masks_weights[mask_str] = weight
            unique_masks_logits[mask_str] = pred
            unique_masks_count[mask_str] = 1

    # Convert dictionary keys back to boolean masks
    unique_keys = sorted(unique_masks_weights.keys())
    unique_masks = [unique_masks_dict[key] for key in unique_keys]
    summed_weights = [unique_masks_weights[key] for key in unique_keys]
    mean_logits = [unique_masks_logits[key] for key in unique_keys]

    return unique_masks, summed_weights, mean_logits #, mean_poolers

In [3]:
import os
import torch
from tqdm.auto import tqdm

input_dir = '../../exps/cosmogrid_4h/best/val_results'
output_dir = '../../exps/cosmogrid_4h/best/val_results_simp'
os.makedirs(output_dir, exist_ok=True)

X = []
images = []
mask_paths = []
mask_idxs = []
preds_all = []
labels_all = []
mask_id_feat_dict = {}


count = 0
filenames = sorted(os.listdir(input_dir))
stats_all = []
total = 0

for filename in tqdm(filenames):
    if os.path.exists(os.path.join(output_dir, filename)):
        continue
    data = torch.load(os.path.join(input_dir, filename))
    image = data['image']
    original_logits = data['original_logits']
    logits = data['logits']
    unique_logits_used = data['unique_logits_used']
    masks = data['masks']
    unique_masks_used = data['unique_masks_used']
    unique_mask_weights_used = data['unique_mask_weights_used']
    unique_losses_used = data['unique_losses_used']
    label = data['label']
    counts = data['counts']
    num_labels = data['num_labels']
    
    unique_masks, summed_weights, mean_logits = sum_weights_for_unique_masks(unique_masks_used, 
                                                                        unique_mask_weights_used, 
                                                                        unique_logits_used)
    
    new_res = {
        'image': image,
        'label': label,
        'logits': logits,
        'unique_masks': unique_masks,
        'summed_weights': summed_weights,
        'mean_logits': mean_logits
    }
    
    torch.save(new_res, os.path.join(output_dir, filename))

  0%|          | 0/1974 [00:00<?, ?it/s]

KeyboardInterrupt: 